# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [7]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

preTrainedModel = VGG19()
preTrainedModel.trainable = False
preTrainedModel.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Create complete Model with Custom output Layer

In [8]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(preTrainedModel)
completeModel.add(Dense(100, activation = "relu"))
# completeModel.add(Dense(75, activation = "relu"))
# completeModel.add(Dense(50, activation = "relu"))
# completeModel.add(Dense(25, activation = "relu"))
completeModel.add(Dense(1, activation = "sigmoid"))
completeModel.summary()
completeModel.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense (Dense)               (None, 100)               100100    
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 143,767,441
Trainable params: 100,201
Non-trainable params: 143,667,240
_________________________________________________________________


/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Orgainze test and training data
Be sure to change file contents

In [10]:
trainingValues = [0] * 1028 + [1] * 1038
testValues = [1] * 160 + [0] * 154
trainImages = keras.utils.image_dataset_from_directory( #Takes given directory and automatically organizes it based on subdirectories
    directory='trainingData',
    labels = trainingValues,
    label_mode='binary',
    batch_size=32,
    image_size=(224, 224))

# Test Images not configured
# testImages = keras.utils.image_dataset_from_directory(
#     directory='testData',
#     labels= testValues,
#     label_mode='binary',
#     batch_size=32,
#     image_size=(224, 224))
# print (trainImages)

Found 2066 files belonging to 2 classes.


## Prediction before training

In [11]:
testPicture = load_img('../deerCrop.jpeg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

2023-11-01 21:00:10.993232: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-11-01 21:00:11.097838: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 292ms/step
[[0.5159187]]


## Train Last two layers

In [12]:
completeModel.fit(trainImages, epochs=10, batch_size=100)

Epoch 1/10


2023-11-01 21:00:13.958088: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


65/65 [==============================] - 21s 317ms/step - loss: 0.6252 - accuracy: 0.8606
Epoch 2/10
65/65 [==============================] - 21s 314ms/step - loss: 0.4311 - accuracy: 0.8887
Epoch 3/10
65/65 [==============================] - 21s 318ms/step - loss: 0.3064 - accuracy: 0.8988
Epoch 4/10
65/65 [==============================] - 21s 314ms/step - loss: 0.2559 - accuracy: 0.9037
Epoch 5/10
65/65 [==============================] - 21s 314ms/step - loss: 0.2286 - accuracy: 0.9100
Epoch 6/10
65/65 [==============================] - 21s 314ms/step - loss: 0.2107 - accuracy: 0.9138
Epoch 7/10
65/65 [==============================] - 21s 314ms/step - loss: 0.1964 - accuracy: 0.9197
Epoch 8/10
65/65 [==============================] - 21s 314ms/step - loss: 0.1846 - accuracy: 0.9240
Epoch 9/10
65/65 [==============================] - 21s 314ms/step - loss: 0.1751 - accuracy: 0.9327
Epoch 10/10
65/65 [==============================] - 21s 314ms/step - loss: 0.1663 - accuracy: 0.9366


## Test Some data

In [ ]:
# score = completeModel.evaluate(testImages)
# print ("loss = ", score[0])
# print ("accuracy = ", score[1])

## See what happens with a deer image

In [26]:
from PIL import Image
testPicture = load_img('../deerCrop.jpeg', target_size=(224,224))
# testPicture = load_img('../deerNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../foxNightCrop2.jpg', target_size=(224,224))
# testPicture = load_img('../possumNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../downloadRabbit.jpeg', target_size=(224,224))
# testPicture = load_img('../rabbitNightCrop1.jpg', target_size=(224,224))
testPicture.show()
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)


# 0 is deer, 1 is fox, 2 is Possum, 3 is rabbit, 4 is squirrel

1/1 [==============================] - 0s 76ms/step
[[0.09450696]]


## Save the model for export

In [25]:
completeModel.save("dogModel.keras")

## Check test data for each animal

In [16]:
# import required module
import os
# assign directory
directory = 'extraTest/testDeer'
 
# iterate over files in
# that directory
# 0 is positives 1 is negativess
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1

print (results)

extraTest/testDeer/1705.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 40ms/step
[[0.05612717]]
extraTest/testDeer/1687.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.66522163]]
extraTest/testDeer/1703.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.156341]]
extraTest/testDeer/1692.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 17ms/step
[[0.3319446]]
extraTest/testDeer/1673.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.28273216]]
extraTest/testDeer/1694.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.10471791]]
extraTest/testDeer/1681.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.01373911]]
extraTest/testDeer/1679.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.18628499]]
extraTest/testDeer/1698.jpg

In [17]:
directory = 'extraTest/testFox'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testFox/3257.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 30ms/step
[[0.03742094]]
extraTest/testFox/3294.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.01116858]]
extraTest/testFox/3287.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.13543014]]
extraTest/testFox/3281.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.43851745]]
extraTest/testFox/3292.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.043624]]
extraTest/testFox/3279.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.00122744]]
extraTest/testFox/3260.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.08028412]]
extraTest/testFox/3273.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.02469733]]
extraTest/testFox/3298.jpg___crop0

In [18]:
directory = 'extraTest/testPossum'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testPossum/1980.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.25039238]]
extraTest/testPossum/1993.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.03633395]]
extraTest/testPossum/1978.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.1060566]]
extraTest/testPossum/1937.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.96108276]]
extraTest/testPossum/1995.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.16824116]]
extraTest/testPossum/1986.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.99656683]]
extraTest/testPossum/1974.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.4710855]]
extraTest/testPossum/1967.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.97289455]]
extraTest/

In [19]:
directory = 'extraTest/testRabbit'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testRabbit/735.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.05891116]]
extraTest/testRabbit/769.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.0988717]]
extraTest/testRabbit/726.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.05990723]]
extraTest/testRabbit/720.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.0207206]]
extraTest/testRabbit/733.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.05124468]]
extraTest/testRabbit/765.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.13332668]]
extraTest/testRabbit/739.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.05107344]]
extraTest/testRabbit/776.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.16063398]]
extraTest/testRabb

In [20]:
directory = 'extraTest/testSquirrel'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testSquirrel/85.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.02923001]]
extraTest/testSquirrel/83.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.01558248]]
extraTest/testSquirrel/118.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.06353188]]
extraTest/testSquirrel/89.jpg___crop02_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.00698067]]
extraTest/testSquirrel/125.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.5893484]]
extraTest/testSquirrel/97.jpg___crop01_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.00309327]]
extraTest/testSquirrel/122.jpg___crop03_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.54048926]]
extraTest/testSquirrel/101.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.07266361]]
extra

In [22]:
directory = 'extraTest/testDog'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testDog/5654.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.6909595]]
extraTest/testDog/5670.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.40364444]]
extraTest/testDog/5667.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9911328]]
extraTest/testDog/5665.jpg___crop00_md_v5a.0.0.ptFlippedImageFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9966979]]
extraTest/testDog/5665.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.99661535]]
extraTest/testDog/5649.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 14ms/step
[[0.66521156]]
extraTest/testDog/5688.jpg___crop00_md_v5a.0.0.ptFlippedImage.jpg
1/1 [==============================] - 0s 13ms/step
[[0.02128867]]
extraTest/testDog/5656.jpg___crop00_md_v5a.0.0.ptFlippedImag

In [23]:
directory = 'extraTest/testCat'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testCat/7051.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.7011072]]
extraTest/testCat/7044.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.98485845]]
extraTest/testCat/7057.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.49273863]]
extraTest/testCat/7048.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.968677]]
extraTest/testCat/7075.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.614914]]
extraTest/testCat/7066.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.9830912]]
extraTest/testCat/7098.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 15ms/step
[[0.06673083]]
extraTest/testCat/7060.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.42483902]]
extraTest/testCat/7073.jpg___crop00_md

In [24]:
directory = 'extraTest/testCoyote'
 
# iterate over files in
# that directory
results = [0,0]
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        testPicture = load_img(f, target_size=(224,224))
        newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
        newPic.save(f[:-4] + "FlippedImage" + f[-4:])
        pictureArray = img_to_array(testPicture)
        pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
        pictureArray = preprocess_input(pictureArray)
        yhat = completeModel.predict(pictureArray)
        print (yhat)
        zhat = yhat.tolist()
        x = zhat[0][0]
        if x > 0.5:
            results[0] += 1
        else:
            results[1] += 1
print (results)

extraTest/testCoyote/58ac0d93-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.01505354]]
extraTest/testCoyote/58b6643a-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.00711395]]
extraTest/testCoyote/5a049387-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 12ms/step
[[0.01964072]]
extraTest/testCoyote/58adc235-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 13ms/step
[[0.18679918]]
extraTest/testCoyote/58ac0d6c-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.00763864]]
extraTest/testCoyote/5a096aec-23d2-11e8-a6a3-ec086b02610b.jpg___crop00_md_v5a.0.0.pt.jpg
1/1 [==============================] - 0s 14ms/step
[[0.0163868]]
extraTest/testCoyote/5a049367-23d2-11e8-a6a3-ec086b02610b.jpg___c

## Code for data manipulation to creaet more data

In [ ]:
# import os
# from PIL import Image
# directory = 'extraTraining/cropped_dog' #Just change directory name for different animal
 
# # iterate over files in
# # that directory
# results = [0,0,0,0]
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         print(f)
#         testPicture = load_img(f, target_size=(224,224))
#         newPic = testPicture.transpose(Image.FLIP_LEFT_RIGHT)
#         newPic.save(f[:-4] + "FlippedImage" + f[-4:])

extraTraining/cropped_dog/5291.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5553.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5756.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5159.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5719.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5116.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5313.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5105.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5300.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5540.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5745.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5688.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5282.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5235.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5670.jpg___crop00_md_v5a.0.0.pt.jpg
extraTraining/cropped_dog/5663.jpg___crop00_md_v5a.0.0.pt.jpg
extraTra